# Calculate residuals, colocate ICESAT1 points

You can calculate the residuals from the track*_all_dzdt.mat files in /Users/home/horganhu/ICESAT_LINK/TAMATA_ICESAT/GLA12_633_DZDT/
```
track0099_all_dzdt.mat
dzdt.track0099.bin2963.grad,
dzdt.track0099.bin2963.dzdt,
dzdt.track0099.bin2963.x,
dzdt.track0099.bin2963.y,
dzdt.track0099.bin2963.z,
dzdt.track0099.bin2963.retide,
dzdt.track0099.bin2963.sdatedays,
dzdt.track0099.bin2963.elev_wgs84_tpxtide,
dzdt.track0099.bin2963.lat,
dzdt.track0099.bin2963.lon,
dzdt.track0099.bin2963.ret
```


```matlab
zp = [x-mean(x); y-mean(y)]'*grad + (t-t0)'*(bindzdt) + mean(z); % Note here we use z here (not elev)

zp=zp';

res = z - zp;     % transient changes (footprint elevation changes corrected for
                % slope and secular change. Note here we use z here (not elev)
                
dz=res + ((t - t0).*bindzdt);   % Elevation change corrected for each segment. This is
                % corrected for gradient. Note, this is per day.
```                

In [77]:
import h5py
import xarray as xr
import glob
import matplotlib.pyplot as plt
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd
from shapely.ops import nearest_points
import numpy as np

In [163]:
def icesat1_alldzdt_todataframe(track):
    """
    this function will import the .mat and arrange into a dataframe
    track = eg track
    """
    
    
    
    path = f'/Users/home/horganhu/ICESAT_LINK/TAMATA_ICESAT/GLA12_633_DZDT/{track}_all_dzdt.mat'
    df = pd.DataFrame({'x':[],
                       'pass_num':[],
                      })
    psys = []
    h = []
    UTCtime = []
    dh = []

    with h5py.File(path, 'r') as f:
        
        bins = list(f['dzdt'][track].keys())
        
        
        #make dataframe
        for b in bins[:2]:
            print(b)
            x_list = list(f['dzdt'][track][b]['x'][0])
            grad = (list(f['dzdt']['track0099']['bin2963']['grad'][0])[0],list(f['dzdt']['track0099']['bin2963']['grad'][1])[0])
            df_temp =  pd.DataFrame({'x': x_list,
                                    'y': list(f['dzdt'][track][b]['y'][0]),
                                    'bin_number': [b]*len( x_list ),
                                    'dzdt': list(f['dzdt'][track][b]['dzdt'][0])*len( x_list ),
                                    'elev_wgs84_tpxtide': list(f['dzdt'][track][b]['elev_wgs84_tpxtide'][0]),
                                    'grad':[grad]*len( x_list ),
                                    'ret': list(f['dzdt'][track][b]['ret'][0])*len( x_list ),
                                    'retide': list(f['dzdt'][track][b]['retide'][0]),
                                    'sdatedays': list(f['dzdt'][track][b]['sdatedays'][0]),
                                    'z': list(f['dzdt'][track][b]['z'][0])
                                   })
            #convert sdatedays UTCtime to a pd.Timestamp object. 
            df_temp['UTCtime'] = df_temp.sdatedays*86400 - (730486+0.5)*86400 # see work_out_utc_time_to_a_timestamp.ipynb for more details
            df_temp['timestamp'] = [pd.Timestamp.utcfromtimestamp(t)+ (pd.Timestamp(2000,1,1,12) - pd.Timestamp(1970,1,1))
                                    for t in df_temp.UTCtime.to_list()]
            df = df.append(df_temp , ignore_index=True )
        
    
    da = df.query("x > -382064.5722209641 & x < -374781.1654740692 & y > -734075.0820404041 & y < -722764.4514729496")
    da.reset_index(drop=True,inplace=True)
    
    points = [Point(xy) for xy in zip(da.x,da.y)]
    gda = gpd.GeoDataFrame(da,geometry=points,crs=3031)

    return df_temp

In [208]:
( np.matmul( np.vstack([df.x.to_numpy()-df.x.mean(), df.y.to_numpy()-df.y.mean()]).T, np.array(df.grad.iloc[0]))
      +
      (( (df.timestamp-df.timestamp.iloc[0]) /  np.timedelta64(1, 'Y'))*df.dzdt.mean() + df.z.mean() ).to_numpy()
     )  

array([3024.43797274, 3017.70526656, 3010.97133713, 3004.24048027,
       3029.64057548, 3022.89532622, 3016.14000485, 3009.38755706,
       3033.12089104, 3026.38409402, 3019.66082411, 3012.9431929 ,
       3006.23236502, 3029.02818594, 3022.28848641, 3015.54929874,
       3008.80710743, 3037.30641463, 3030.62739302, 3023.95650762,
       3017.2902405 , 3010.62328832, 3036.50852623, 3029.78001726,
       3023.05655662, 3016.3448399 , 3009.63190251, 3031.68673389,
       3024.96052912, 3018.22288757, 3011.48307198, 3004.73650538,
       3026.58649294, 3019.84416817, 3013.10213451, 3006.36062251,
       3025.82627769, 3019.09782589, 3012.36044393, 3005.62653027,
       3034.02995168, 3027.34998588, 3020.6791214 , 3014.01065727,
       3007.34694721, 3028.02484245, 3021.35730675, 3014.69750633,
       3008.04348736, 3026.53360278, 3019.79294362, 3013.06551288,
       3006.35210007, 3032.11618012, 3025.42133829, 3018.71519163,
       3011.99844107, 3005.26989744, 3039.46127644, 3032.76133

In [209]:
# calculate residual

# ? grad
# ? t0

df['zp'] = ( np.matmul( np.vstack([df.x.to_numpy()-df.x.mean(), df.y.to_numpy()-df.y.mean()]).T, np.array(df.grad.iloc[0]))
      +
      (( (df.timestamp-df.timestamp.iloc[0]) /  np.timedelta64(1, 'Y'))*df.dzdt.mean() + df.z.mean() ).to_numpy()
     )       # Note here we use z here (not elev)

df.res = df.z - df.zp;     # transient changes (footprint elevation changes corrected for
                # slope and secular change. Note here we use z here (not elev)

dz = df.res + ( (df.timestamp-df.timestamp.iloc[0]) /  np.timedelta64(1, 'Y'))*df.dzdt.iloc[0];   # Elevation change corrected for each segment. This is
                # corrected for gradient. Note, this is per year.

/Users/home/whitefar/miniconda3/envs/remote_sensing/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  # This is added back by InteractiveShellApp.init_path()


In [165]:
df = df0099

In [147]:
track = 'track0099'
b='bin10000'
with h5py.File(path, 'r') as f:
    x_list = list(f['dzdt'][track][b]['x'][0])
    a = [(list(f['dzdt']['track0099']['bin2963']['grad'][0])[0],list(f['dzdt']['track0099']['bin2963']['grad'][1])[0])]*len( x_list )
a


[(0.05849730400946533, -0.004219829152570147),
 (0.05849730400946533, -0.004219829152570147),
 (0.05849730400946533, -0.004219829152570147),
 (0.05849730400946533, -0.004219829152570147),
 (0.05849730400946533, -0.004219829152570147),
 (0.05849730400946533, -0.004219829152570147),
 (0.05849730400946533, -0.004219829152570147),
 (0.05849730400946533, -0.004219829152570147),
 (0.05849730400946533, -0.004219829152570147),
 (0.05849730400946533, -0.004219829152570147),
 (0.05849730400946533, -0.004219829152570147),
 (0.05849730400946533, -0.004219829152570147),
 (0.05849730400946533, -0.004219829152570147),
 (0.05849730400946533, -0.004219829152570147),
 (0.05849730400946533, -0.004219829152570147),
 (0.05849730400946533, -0.004219829152570147),
 (0.05849730400946533, -0.004219829152570147),
 (0.05849730400946533, -0.004219829152570147),
 (0.05849730400946533, -0.004219829152570147),
 (0.05849730400946533, -0.004219829152570147),
 (0.05849730400946533, -0.004219829152570147),
 (0.058497304

### from load_matlab_icesat1_structures we see the tracks over the channel with  the most data
track number_of_datapoints
track0015 0
track1331 68
track0043 0
track0197 68
track1303 68
track0351 68
track0057 0
track0407 68
track0155 68
track0127 68
track0183 68
track0365 68
track0337 68
track0099 753
track0295 68
track0323 68
track0211 549
track0113 68
track0379 68
track0085 193
track0267 68
track0071 0
track0169 68
track1289 68
track0029 0
track1345 68
track0239 68
track1275 68
track0225 68
track0001 0
track0393 68
track0141 68
track0253 68
track0309 68
track1317 68
track0281 68

In [164]:
df0099 = icesat1_alldzdt_todataframe('track0099')

bin10000
bin100000


In [156]:
df0099.keys()

Index(['x', 'y', 'bin_number', 'dzdt', 'elev_wgs84_tpxtide', 'grad', 'ret',
       'retide', 'sdatedays', 'z'],
      dtype='object')

In [160]:
df0099.ret

0     100.0
1     100.0
2     100.0
3     100.0
4     100.0
      ...  
66    100.0
67    100.0
68    100.0
69    100.0
70    100.0
Name: ret, Length: 71, dtype: float64

In [108]:
df = df0099.copy()

In [150]:
df0099['bin10000'].keys()

KeyError: 'bin10000'

In [47]:
len(df0099['bin99999']['z'])

71

In [ ]:
df0099['bin99999']['sdatedays']

In [26]:
with h5py.File(path, 'r') as f:
    bins = list(f['dzdt']['track0099'].keys())

In [27]:
bins

['bin10000',
 'bin100000',
 'bin100001',
 'bin100002',
 'bin100003',
 'bin100004',
 'bin100005',
 'bin100006',
 'bin100007',
 'bin100008',
 'bin100009',
 'bin10001',
 'bin100010',
 'bin100011',
 'bin100012',
 'bin100013',
 'bin100014',
 'bin100015',
 'bin100016',
 'bin100017',
 'bin100018',
 'bin100019',
 'bin10002',
 'bin100020',
 'bin100021',
 'bin100022',
 'bin100023',
 'bin100024',
 'bin100025',
 'bin100026',
 'bin100027',
 'bin100028',
 'bin100029',
 'bin10003',
 'bin100030',
 'bin100031',
 'bin100032',
 'bin100033',
 'bin100034',
 'bin100035',
 'bin100036',
 'bin100037',
 'bin100038',
 'bin100039',
 'bin10004',
 'bin100040',
 'bin100041',
 'bin100042',
 'bin100043',
 'bin100044',
 'bin100045',
 'bin100046',
 'bin100047',
 'bin100048',
 'bin100049',
 'bin10005',
 'bin100050',
 'bin100051',
 'bin100052',
 'bin100053',
 'bin100054',
 'bin100055',
 'bin100056',
 'bin100057',
 'bin100058',
 'bin100059',
 'bin10006',
 'bin100060',
 'bin100061',
 'bin100062',
 'bin100063',
 'bin100064',

In [32]:
b = bins[-5]
with h5py.File(path, 'r') as f:
    print(f['dzdt']['track0099'][b]['x'][0][:5])

[264119.53128335 264014.06443773 263908.66451467 263803.318851
 264208.5821295 ]


In [10]:
track = '0099'
path = f'/Users/home/horganhu/ICESAT_LINK/TAMATA_ICESAT/GLA12_633_DZDT/track{track}_all_dzdt.mat'

In [11]:
df = pd.DataFrame({'x':[],
                   'pass_num':[],
                  })
psys = []
h = []
UTCtime = []
dh = []

with h5py.File(path, 'r') as f:
    #add x location
    psx_l = list(f['antarctica'][track]['psx'])
    for p in psx_l:
        df_temp = pd.DataFrame({'x': list(f['antarctica'][track]['psx'][p][0]),
                                'pass_num': [p]*len(list(f['antarctica'][track]['psx'][p])[0]) })
        df = df.append(df_temp , ignore_index=True )

#     #add y location
#     psy_l = list(f['antarctica'][track]['psy'])
#     for p in psy_l:
#         psys.extend( list(f['antarctica'][track]['psy'][p][0]) )

#     #add height
#     h_l = list(f['antarctica'][track]['elev_wgs84_retide'])
#     for p in h_l:
#         h.extend( list(f['antarctica'][track]['elev_wgs84_retide'][p][0]) )

#     #add time
#     t_l = list(f['antarctica'][track]['UTCTime'])
#     for p in t_l:
#         UTCtime.extend( list(f['antarctica'][track]['UTCTime'][p][0]) )

#     #add delta_h (not sure what it is)
#     dh_l = list(f['antarctica'][track]['delta_h'])
#     for p in dh_l:
#         dh.extend( list(f['antarctica'][track]['delta_h'][p][0]) )

KeyError: "Unable to open object (object 'antarctica' doesn't exist)"

In [3]:
df = icesat1todataframe('track0099')

In [12]:
len([12,3])

2

In [ ]:
zp = [x-mean(x); y-mean(y)]'*grad + (t-t0)'*(bindzdt) + mean(z); % Note here we use z here (not elev)

zp=zp';

res = z - zp;     % transient changes (footprint elevation changes corrected for
                % slope and secular change. Note here we use z here (not elev)

dz=res + ((t - t0).*bindzdt);   % Elevation change corrected for each segment. This is
                % corrected for gradient. Note, this is per day.